In [1]:
import sys
sys.path.append("../") # go to parent dir
# imports
import simpy
from loguru import logger
import time
import numpy as np
import pandas as pd
from cluster_simulator.utils import convex_hull
from cluster_simulator.cluster import Cluster, Tier, EphemeralTier, bandwidth_share_model, compute_share_model, get_tier, convert_size
from cluster_simulator.phase import DelayPhase, ComputePhase, IOPhase
from cluster_simulator.application import Application
from cluster_simulator.analytics import display_run
from cluster_simulator.ephemeral_placement import ClusterBlackBox
from cluster_simulator.placement_optimizer import PlacementOptimizer


### 6.3 Exemples de Contraintes en Code

L'un des aspects les plus importants de notre implémentation est la manière dont nous avons formalisé les contraintes du problème d'ordonnancement. Utilisant la bibliothèque `PuLP`, nous avons pu traduire les contraintes mathématiques en expressions de code compréhensibles et efficaces.

Par exemple, pour s'assurer qu'une phase ne commence qu'après la fin de la phase précédente dans la même tâche, nous utilisons la contrainte suivante :

```python
if i > 0:
    prob += x[job, i] >= y[job, i - 1], f"StartAfterPrev_{job}_{i}"
```

Cette expression utilise des variables de décision \( x \) et \( y \) qui représentent respectivement le temps de début et de fin de chaque phase pour chaque tâche.

Un autre exemple de contrainte est celui qui calcule la durée d'une phase en fonction de la somme des volumes de lecture et d'écriture et de la bande passante du tier de stockage choisi :

```python
total_volume = phase['read_volume'] + phase['write_volume']
bandwidth = tier0_model(phase) if z[job, i, 0] == 1 else tier1_model(phase)
prob += y[job, i] == x[job, i] + total_volume / bandwidth, f"EndTime_{job}_{i}"
```

Nous avons également ajouté des contraintes pour gérer la sélection des tiers de stockage et pour s'assurer que la bande passante globale n'excède pas les limites des tiers respectifs.

Ces exemples illustrent comment notre implémentation en Python, appuyée par la bibliothèque `PuLP`, permet de formaliser un ensemble complexe de contraintes de manière claire et efficace. Ce formalisme facilite non seulement la résolution du problème, mais aussi l'expérimentation avec différentes configurations et objectifs d'optimisation.





Bien sûr, voici un récapitulatif des différents types de contraintes que nous avons abordés dans les exemples de code précédents :

1. **Contraintes de séquencement entre les phases d'un même job** : Ces contraintes garantissent que chaque phase d'un job commence après la fin de la phase précédente.

    ```python
    prob += x[job, i] >= y[job, i - 1], f"StartAfterPrev_{job}_{i}"
    ```

2. **Contraintes de durée de phase** : Ces contraintes sont utilisées pour calculer le temps nécessaire pour exécuter une phase en fonction de son volume total de données (lecture + écriture) et de la bande passante du tier utilisé.

    ```python
    total_volume = phase['read_volume'] + phase['write_volume']
    bandwidth = tier0_model(phase) if z[job, i, 0] == 1 else tier1_model(phase)
    prob += y[job, i] == x[job, i] + total_volume / bandwidth, f"EndTime_{job}_{i}"
    ```

3. **Contraintes de choix de tier** : Ces contraintes garantissent qu'un seul tier est utilisé pour chaque phase.

    ```python
    prob += z[job, i, 0] + z[job, i, 1] == 1, f"SingleTier_{job}_{i}"
    ```

4. **Contraintes d'exécution en parallèle ou en séquence** : Dans le cas où les jobs doivent être exécutés en séquence, une contrainte supplémentaire serait ajoutée pour garantir que le job suivant ne commence qu'après la fin du job précédent.

    ```python
    if sequential:
        prob += x['job2', 0] >= y['job1', len(job1_phase_features) - 1], "Sequential_Jobs"
    ```

Ces contraintes forment la base de notre modèle d'ordonnancement et d'allocation des ressources, permettant une flexibilité dans la prise en compte des différentes dimensions du problème, telles que les volumes de données, les choix de tiers de stockage, et les exigences de séquencement des tâches.

La contrainte sur le partage de la bande passante lorsque deux ou plusieurs phases utilisent le même tier peut être cruciale pour assurer un fonctionnement optimal du système. Cette contrainte garantit que la somme des bandes passantes allouées aux différentes phases n'excède pas la capacité maximale du tier concerné. 

Supposons que la capacité maximale de bande passante pour les tiers 0 et 1 soit respectivement \(BW_{\text{max, tier0}}\) et \(BW_{\text{max, tier1}}\).

La variable de décision \(w_{(i, j, k)}(t)\) représente la bande passante attribuée à la phase \(j\) de la tâche \(i\) lorsqu'elle utilise le tier \(k\) au temps \(t\). La contrainte de partage de la bande passante peut alors être exprimée comme suit :

\[
\sum_{i,j} w_{(i, j, k)}(t) \leq BW_{\text{max, tier}_k}, \quad \forall t, k
\]

Dans le code Python utilisant PuLP, cela pourrait être implémenté comme :

```python
# Bandwidth capacity constraints
for t in time_range:
    for k in [0, 1]:  # For each tier
        prob += lpSum(w[i, j, k, t] for i, phase_list in phases.items() for j in range(len(phase_list))) <= max_bandwidth[k], f"MaxBandwidth_Tier{k}_Time{t}"
```

Ici, `max_bandwidth` serait un dictionnaire contenant les capacités maximales de bande passante pour chaque tier, et `time_range` serait la plage de temps sur laquelle s'étend le problème.

Vous avez tout à fait raison. La capacité maximale de la bande passante, représentée par `max_bandwidth[k]`, peut être déterminée en utilisant un modèle de régression qui prend en compte les caractéristiques de chaque phase. Ce modèle de régression pourrait être basé sur différents facteurs, tels que le volume de lecture/écriture, le nombre de nœuds, le type de modèle d'E/S, etc.

Dans ce cas, la contrainte de bande passante maximale peut être dynamiquement mise à jour en fonction des caractéristiques de la phase. Par exemple :

```python
# Bandwidth capacity constraints
for t in time_range:
    for k in [0, 1]:  # For each tier
        # Using the regression model to get the maximum bandwidth for the tier based on phase features
        max_bandwidth_k = tier_model[k](phase_features)
        
        prob += lpSum(w[i, j, k, t] for i, phase_list in phases.items() for j in range(len(phase_list))) <= max_bandwidth_k, f"MaxBandwidth_Tier{k}_Time{t}"
```

Ici, `tier_model[k]` représente le modèle de régression pour le tier \( k \), et `phase_features` serait un ensemble de caractéristiques pertinentes pour la phase considérée. Ce modèle serait utilisé pour prédire la bande passante maximale possible pour un tier donné en fonction des caractéristiques de la phase en cours d'exécution.





You're absolutely correct; I apologize for the oversight. If the `max_bandwidth` for each tier is dynamically determined using a regression model based on the phase features, the constraint should be formulated within the loop where each phase is accessed. Here is how you might do it:

```python
# Bandwidth capacity constraints
for t in time_range:
    for k in [0, 1]:  # For each tier
        # Summation of bandwidth used by all phases at time t for tier k should not exceed the max bandwidth
        prob += lpSum(
            w[i, j, k, t] for i, phase_list in phases.items() for j, phase in enumerate(phase_list)
        ) <= tier_model[k](phase), f"MaxBandwidth_Tier{k}_Time{t}"
```
```python
# Contraintes sur la capacité de la bande passante
for t in time_range:
    for k in [0, 1]:  # Pour chaque niveau de stockage (tier)
        # La somme de la bande passante utilisée par toutes les phases au temps t pour le niveau k 
        # ne doit pas dépasser la bande passante maximale pour ce niveau.
        # La fonction tier_model[k](phase) évalue la bande passante maximale pour le niveau k 
        # en fonction des caractéristiques de la phase en cours.
        prob += lpSum(
            w[i, j, k, t] for i, phase_list in phases.items() for j, phase in enumerate(phase_list)
        ) <= tier_model[k](phase), f"MaxBandwidth_Tier{k}_Time{t}"




```

La contrainte assure que la bande passante totale consommée par toutes les phases à tout moment \( t \) ne dépasse pas la bande passante maximale pour le niveau de stockage \( k \). Cette bande passante maximale est déterminée de manière dynamique en utilisant le modèle de régression `tier_model[k](phase)` sur les caractéristiques de la phase en cours.


Cette approche vous permet de tenir compte des variations potentielles de la bande passante en fonction de divers facteurs, rendant ainsi votre modèle plus adaptatif et robuste.

Notez que cette approche nécessite de discrétiser le temps en plusieurs pas de temps, ce qui peut augmenter considérablement la taille du problème. Le choix de la taille du pas de temps dépendra donc d'un compromis entre précision et complexité de calcul.




Absolument, il est plus logique de diviser par \( n_k(t) \) uniquement si \( n_k(t) > 0 \). Vous pouvez le faire en ajoutant une condition dans votre contrainte. Voici comment cela pourrait être fait :

```python
# Contraintes sur le partage de la capacité de la bande passante
for t in time_range:
    for k in [0, 1]:  # Pour chaque niveau de stockage (tier)
        # Calcul du nombre de phases utilisant le tier k au temps t
        n_k_t = lpSum(u[i, j, k, t] for i, phase_list in phases.items() for j, phase in enumerate(phase_list))

        # La somme de la bande passante utilisée par toutes les phases au temps t pour le niveau k
        # ne doit pas dépasser la bande passante maximale pour ce niveau.
        max_bandwidth_k = tier_model[k](phase)
        prob += lpSum(
            w[i, j, k, t] for i, phase_list in phases.items() for j, phase in enumerate(phase_list)
        ) <= max_bandwidth_k, f"MaxBandwidth_Tier{k}_Time{t}"

        # Si n_k_t > 0, alors on divise la bande passante maximale par n_k_t
        prob += lpSum(
            w[i, j, k, t] for i, phase_list in phases.items() for j, phase in enumerate(phase_list)
        ) <= max_bandwidth_k / (n_k_t + (n_k_t == 0)), f"SharedBandwidth_Tier{k}_Time{t}"
```


Dans le contexte des systèmes HPC (High-Performance Computing), l'ordonnancement et l'allocation des ressources sont des problématiques cruciales. Ces défis sont particulièrement importants lorsque l'on considère des workflows complexes, composés de multiples tâches avec des dépendances et des contraintes diverses, telles que la bande passante, les volumes de lecture et d'écriture, et les différents niveaux de stockage (tiers). La Programmation Linéaire Mixte en Nombres Entiers (MILP) offre une méthode mathématique rigoureuse pour modéliser et résoudre ces types de problèmes. 

L'approche MILP est justifiée par sa capacité à gérer un grand nombre de contraintes linéaires et non-linéaires, tout en fournissant des solutions optimales ou quasi-optimales. Contrairement aux méthodes heuristiques, qui ne garantissent pas toujours une solution optimale, MILP assure que la solution obtenue est la meilleure possible selon la fonction objectif, sous réserve des contraintes définies. De plus, des outils logiciels tels que PuLP permettent une implémentation assez simple et directe des modèles MILP, tout en offrant des temps de résolution rapides pour des problèmes de taille modérée. 

En somme, l'approche MILP est particulièrement bien adaptée pour les environnements HPC où la précision, l'efficacité et la capacité à gérer des contraintes multiples sont essentielles. Elle permet une modélisation fine des différentes facettes du problème et offre une méthode robuste et fiable pour trouver des solutions optimales, ce qui en fait un choix judicieux pour notre système de recommandation destiné à optimiser la gestion des ressources dans les applications HPC.

Je voudrais maintenant compléter le rapport avec une partie dédiée à l'analyse bayésienne. Fondamentalement, le résultat de recommandation premier est obtenu avec la résolution du problème MILP. Il permet d'obtenir un choix initial de tiers de stockage pour chaque phase de chaque job. L'analyse Bayésienne de mettre bout à bout un simulateur d'execution à qui on fournit un vecteur d'affectation de tiers à des phases et il nous retourne une timeline de l'exécution du workflow, et par conséquence sa durée qu'on cherche à optimiser. l'idée est d'utiliser la solution PuLP en réponse lorsque l'utilisateur invoque le système de recommendation. Puis de laisser la recherche bayésienne perturber légèrement le vecteur d'attribution des tiers aux phases des tâches pour explorer les solutions voisines, concurrentes, et ainsi établir un comparatif et pouvoir qualifier l'optimalité de la solution trouvé par PuLP. Après, la recherche bayésienne utilisant BBO (black box optimizer) ou Optuna risque d'itérer à plusieurs reprises sur des solutions de placements, qu'il injecte dans le simulateur d'exécution pour évaluer la performance (temps d'exécution du workflow), risque de prendre un peu plus de temps. En sortie, des graphiques et des analytiques comparatives sur la qualité de la solution trouvée sont prévues. 
propose moi en complément au rapport, une organisation et quelques paragraphe pour exprimer ces idée. Tu peux poser des question pour obtenir des précisions et procéder par parties.



Setting the Cluster:
- with one tier level (HDD)
- with an ephemeral tier:
 

In [5]:
# sim env an data
env = simpy.Environment()
data = simpy.Store(env)
# tier perfs
nvram_bandwidth = {'read':  {'seq': 800, 'rand': 800},
                        'write': {'seq': 400, 'rand': 400}}
ssd_bandwidth = {'read':  {'seq': 200, 'rand': 200},
                    'write': {'seq': 100, 'rand': 100}}
hdd_bandwidth = {'read':  {'seq': 80, 'rand': 80},
                    'write': {'seq': 40, 'rand': 40}}

# registering Tiers
hdd_tier = Tier(env, 'HDD', bandwidth=hdd_bandwidth, capacity=1e12)
ssd_tier = Tier(env, 'SSD', bandwidth=ssd_bandwidth, capacity=200e9)
nvram_tier = Tier(env, 'NVRAM', bandwidth=nvram_bandwidth,
                        capacity=10e9)
# registering Ephemeral Tier
bb = EphemeralTier(env, name="BB", persistent_tier=hdd_tier,
                        bandwidth=nvram_bandwidth, capacity=10e9)

# Define the cluster with 1 persistent and 1 ephemeral
cluster = Cluster(env, tiers=[hdd_tier, ssd_tier], ephemeral_tier=bb)

# define app
read = [1e9, 0]
compute = [0,  10]
write = [0, 5e9]
app1 = Application(env, compute=compute, read=read, write=write, data=data)

# placement
placement = [0, 0]
use_bb = [False, False]

In [6]:
# logger
# sim env an data
data = simpy.Store(env)
logger.remove()
# Simple app: read 1GB -> compute 10s -> write 5GB
compute = [5790,  5960]
read = [0, 0]
write = [1.5e9, 5e9]
placement = [0, 1]
use_bb = [False, False]
app = Application(env, compute=compute, read=read, write=write, data=data)
env.process(app.run(cluster, placement=placement, use_bb=use_bb))
env.run()
print(f"application duration = {app.get_fitness()}")
fig = display_run(data, cluster, width=800, height=900)
fig.show()

application duration = 257.5


Behavior for no SBB app

In [ ]:
p1 = PlacementOptimizer(env, data, cluster, [app1])
fig = p1.display_placement(placement + [0, 0])
fig.show()

Selective SBB per dataset

In [ ]:
p1 = PlacementOptimizer(env, data, cluster, [app1])
fig = p1.display_placement(placement + [0, 1])
fig.show()

What about prefetch ? (buffering read datasets)

In [ ]:
# logger
# sim env an data
data = simpy.Store(env)
logger.remove()
# Simple app: read 1GB -> compute 10s -> write 5GB
compute = [0,  10]
read = [1e9, 0]
write = [0, 5e9]
placement = [0, 0]
use_bb = [True, False]
app = Application(env, compute=compute, read=read, write=write, data=data)
env.process(app.run(cluster, placement=placement, use_bb=use_bb))
env.run()
print(f"application duration = {app.get_fitness()}")
fig = display_run(data, cluster, width=800, height=900)
fig.show()

Burst Buffer Saturation
 - will choose bb_size < 5GB (max used size)
 

In [ ]:
# registering Ephemeral Tier
bb = EphemeralTier(env, name="BB", persistent_tier=hdd_tier,
                        bandwidth=nvram_bandwidth, capacity=4.5e9)

# Define the cluster with 1 persistent and 1 ephemeral
cluster = Cluster(env, tiers=[hdd_tier], ephemeral_tier=bb)
# logger
# sim env an data
data = simpy.Store(env)
logger.remove()
# Simple app: read 1GB -> compute 10s -> write 5GB
compute = [0,  10]
read = [1e9, 0]
write = [0, 5e9]
placement = [0, 0]
use_bb = [False, True] # enabling ephemeral tier for second dataset
app = Application(env, compute=compute, read=read, write=write, data=data)
env.process(app.run(cluster, placement=placement, use_bb=use_bb))
env.run()
print(f"application duration = {app.get_fitness()}")
fig = display_run(data, cluster, width=800, height=900)
fig.show()

- eviction occurs
- buffer saturation should be avoided:
  - only newly copied data to lower tier diminishes the amount of dirty data
  - we can only evict clean data
  - in terms of simulations, it shorten the time interval and the loop become extremly slow (~inf)
- the focus is not on studying what happens in saturation mode, but to size relevantly the BB

Optimizing BB Size (flavor)

In [ ]:
import simpy
from loguru import logger
import time
import numpy as np
import pandas as pd
import time, os
from itertools import chain
from cluster_simulator.cluster import Cluster, Tier, EphemeralTier, bandwidth_share_model, compute_share_model, get_tier, convert_size
from cluster_simulator.phase import DelayPhase, ComputePhase, IOPhase
from cluster_simulator.application import Application
from cluster_simulator.analytics import display_run
# imports for surrogate models
from sklearn.gaussian_process import GaussianProcessRegressor
from bbo.optimizer import BBOptimizer
# from bbo.optimizer import timeit
from bbo.heuristics.surrogate_models.next_parameter_strategies import expected_improvement

# imports for genetic algorithms
from bbo.heuristics.genetic_algorithm.selections import tournament_pick
from bbo.heuristics.genetic_algorithm.crossover import double_point_crossover
from bbo.heuristics.genetic_algorithm.mutations import mutate_chromosome_to_neighbor
from loguru import logger
from cluster_simulator.ephemeral_placement import ClusterBlackBox
logger.remove()
cbb = ClusterBlackBox()
PARAMETER_SPACE = cbb.parameter_space
# combinations are self.n_tiers ** sum(self.ios) + 2**sum(self.ios)
NBR_ITERATION = 1  # cbb.n_tiers ** sum(cbb.ios)

np.random.seed(5)
bbopt = BBOptimizer(black_box=cbb,
                    heuristic="surrogate_model",
                    max_iteration=NBR_ITERATION,
                    initial_sample_size=2,
                    parameter_space=PARAMETER_SPACE,
                    next_parameter_strategy=expected_improvement,
                    regression_model=GaussianProcessRegressor)
start_time = time.time()
bbopt.optimize()
print("-----------------")
print(f"Total number of iterations : {NBR_ITERATION}")
print(f"{(time.time() - start_time)} seconds spent for finding solution")
print("-----------------")
bbopt.summarize()
print(f"Fitness history : {bbopt.history['fitness']}")
#cbb.save_experiment(filename = "flavor_optim", save=True)

In [ ]:
import numpy as np
import pandas as pd
import os
!pip3 install pickle5
import pickle5 as pickle5

pathfile = os.path.join(os.getcwd(), "flavor_optim")
df = pd.read_pickle(pathfile)
df.head(2)
#list(df.columns)

Plotting Workflow duration Vs BB Size : efficient frontier


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
colors = px.colors.qualitative.Plotly
fig = go.Figure()
fig.add_traces(go.Scatter(x=df['BB_size'], y = df['Fitness'], 
                          text = ["trial_index=" + str(i) for i in list(df.index)],
                          mode = 'markers', line=dict(color=colors[0])))
fig.update_xaxes(title_text='Size of BB in bytes (B = 1e9 = GB)')
fig.update_yaxes(title_text='Workflow duration in seconds')
fig.show()

- each point is a placement configuration:
  - for each dataset:
    - the tier where it should be placed
    - the use or not of the BB to prefetch/buffer 

Show some samples

In [ ]:
# indicate the number of trial
cbb.display_placement(df.loc[29]["Param"])

Efficient frontier / Pareto frontier

In [ ]:
from cluster_simulator.utils import convex_hull
points = []
for (xi, yi) in zip(df['BB_size'].tolist(), (df['Fitness']).tolist()):
    points.append((xi,yi))
lower = convex_hull(points)
fig.add_traces(go.Scatter(x=np.array([low[0] for low in lower]),
                            y=np.array([low[1] for low in lower])))
fig.show()

Recommandation are elements from efficient/pareto frontier

In [ ]:
rec_trials = [29, 31, 4, 6, 129]
df.loc[rec_trials][df.columns[1:]]

Display individual points (placement recommandation)

In [ ]:
fig1 = cbb.display_placement(placement=bbopt.best_parameters_in_grid)
fig1.show()